![GenAI Banner](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)


<p><font size="5" color='grey'> <b>
Agenten
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>Umgebung einrichten</font> </br></p>
!uv pip install --system --prerelease allow -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN', 'WEATHER_API_KEY', 'SERPAPI_API_KEY', 'TAVILY_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

# 1 | Unterschied LLM vs Agenten

Der Unterschied zwischen einem LLM und einem Agenten ist wie der Unterschied zwischen einem Buch und einem Menschen mit Zugang zu Werkzeugen und Internet. Diese Unterscheidung ist entscheidend für das Verständnis moderner KI-Anwendungen.

**Problem: Wo LLMs an ihre Grenzen stoßen**

Ein Large Language Model ist im Grunde ein sehr fortgeschrittenes Textverarbeitungsystem, das auf Basis seiner Trainingsdaten antwortet. Diese Begrenzung führt zu vier grundlegenden Problemen:

+ **Veraltete Informationen**: LLMs kennen nur Daten bis zu ihrem Trainingsstichtag. Fragen Sie nach aktuellen Ereignissen, Börsenkursen oder dem heutigen Wetter, erhalten Sie veraltete oder keine Informationen.

+ **Ungenaue Berechnungen**: Während LLMs einfache Mathematik verstehen, sind sie bei komplexeren Berechnungen oft ungenau. Sie "raten" Ergebnisse basierend auf Mustern, anstatt tatsächlich zu rechnen.

+ **Keine externe Interaktion**: Ein LLM kann nicht im Internet suchen, APIs aufrufen oder Dateien lesen. Es ist vollständig isoliert von der Außenwelt.

+ **Fehlende Planung**: LLMs können keine mehrstufigen Prozesse durchführen, bei denen das Ergebnis eines Schritts den nächsten beeinflusst.

<br>

**Lösung: Agenten erweitern LLMs**

Ein Agent löst diese Probleme durch drei Schlüsselkomponenten: Er nutzt das LLM als "Gehirn" für Sprachverständnis und Reasoning, erweitert es aber um Tools für externe Interaktionen und implementiert einen iterativen Denkprozess.

+ **Aktuelle Daten durch Tools**: Agenten können über spezielle Werkzeuge auf aktuelle Informationen zugreifen - von Wetterapis bis zu Internetsuchmaschinen.

+ **Präzise Berechnungen**: Statt zu raten, verwenden Agenten Rechner-Tools für exakte mathematische Operationen.

+ **Unbegrenzte Erweiterbarkeit**: Neue Fähigkeiten entstehen durch neue Tools - von Datenbankzugriff bis zu Bildbearbeitung.

+ **Transparente Planung**: Der Denkprozess des Agenten ist sichtbar und nachvollziehbar, was Vertrauen und Debugging ermöglicht.

# 2 | Direkter Vergleich
---

## 2.1 Setup und Tools

Bevor wir vergleichen können, müssen wir die notwendigen Tools für unseren Agenten definieren. Diese Tools repräsentieren die erweiterten Fähigkeiten, die einem einfachen LLM fehlen.



In [ ]:
# Installationen
!uv pip install --system --prerelease allow -q google-search-results

In [ ]:
# Einfache Tools definieren
from langchain_core.tools import Tool
from langchain_community.utilities.serpapi import SerpAPIWrapper
import requests

def simple_calculator(expression):
    """Einfacher Rechner - Das kann ein LLM oft nicht präzise"""
    try:
        # Sicherheitscheck
        if any(x in expression for x in ['import', 'exec', '__']):
            return "Unsichere Operation"
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "Berechnungsfehler"

def get_weather(city):
    """Aktuelle Wetterdaten - Das kann ein LLM GAR NICHT"""
    # Vereinfachte Demo-Version
    return f"🌤️ Aktuelles Wetter in {city}: 22°C, sonnig (Demo-Version)"

# Tool-Liste erstellen
serpapi = SerpAPIWrapper()

tools = [
    Tool(
        name="internet_search",
        func=serpapi.run,
        description="🌐 INTERNETSUCHE - Aktuelle Informationen finden (was LLM NICHT kann)"
    ),
    Tool(
        name="calculator",
        func=simple_calculator,
        description="🔢 RECHNER - Präzise Berechnungen (was LLM oft falsch macht)"
    ),
    Tool(
        name="weather",
        func=get_weather,
        description="🌤️ WETTER - Echtzeitdaten abrufen (was LLM unmöglich ist)"
    )
]

print("✅ Tools definiert:")
for tool in tools:
    print(f"   • {tool.name}: {tool.description}")

## 2.2 Vergleichstest

Wir verwenden eine Frage, die sowohl aktuelle Daten als auch eine Berechnung erfordert, um die Grenzen eines LLMs und die Stärken eines Agenten zu demonstrieren.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import time

# LLM Setup
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

# Test-Frage die Grenzen aufzeigt
test_question = "Wie ist das Wetter in Berlin und was ist 2847 * 1923?"

mprint("### 🧪 VERGLEICHSTEST")
mprint("---")
mprint(f"**Frage:** {test_question}")
print()

# 1. EINFACHES LLM PROBIEREN
mprint("### 1️⃣ EINFACHES LLM:")
mprint("---")

start_time = time.time()
llm_response = llm.invoke(test_question)
llm_time = time.time() - start_time

mprint(f"**Antwort:** {llm_response.content}")
mprint(f"**Zeit:** {llm_time:.2f}s")
print()
mprint("### ❌ PROBLEME:")
mprint("+ Keine aktuellen Wetterdaten")
mprint("+ Berechnung möglicherweise ungenau")
mprint("+ Kann nicht überprüfen ob Antwort stimmt")
print()

# 2. AGENT MIT TOOLS
mprint("### 2️⃣ AGENT MIT TOOLS:")
mprint("---")

# Agent erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Agent. Nutze Tools für aktuelle Daten und Berechnungen."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Zeigt Denkprozess
    return_intermediate_steps=True
)

start_time = time.time()
agent_response = agent_executor.invoke({"input": test_question})
agent_time = time.time() - start_time

mprint(f"**Antwort:**  {agent_response['output']}")
mprint(f"**Zeit:** {agent_time:.2f}s")
print()
mprint("### ✅ VORTEILE:")
mprint("+ Aktuelle Wetterdaten abgerufen")
mprint("+ Präzise Berechnung durchgeführt")
mprint("+ Transparenter Denkprozess sichtbar")
mprint("+ Schritte nachvollziehbar")


## 2.3 Unterschiede



Die Unterschiede zwischen LLM und Agent lassen sich in fünf Kernbereichen zusammenfassen, die den Paradigmenwechsel von statischer zu dynamischer KI verdeutlichen.

**Vergleich der Fähigkeiten:**

| Aspekt | Einfaches LLM | Agent |
|--------|---------------|-------|
| **Aktuelle Daten** | ❌ Nur Trainingsdaten | ✅ Über Tools |
| **Berechnungen** | ⚠️ Oft ungenau | ✅ Präzise Tools |
| **Externe APIs** | ❌ Unmöglich | ✅ Beliebig erweiterbar |
| **Transparenz** | 🔒 Verborgen | 👁️ Sichtbar |
| **Erweiterbarkeit** | ❌ Statisch | ✅ Modular |



**🎯 FAZIT:**
- **Agent = LLM + Tools + Reasoning**
- ➡️ Aus reaktiv wird proaktiv
- ➡️ Aus statisch wird dynamisch
- ➡️ Aus isoliert wird vernetzt


# 3 | Anatomie eines Agenten
---



Um Agenten effektiv einsetzen zu können, muss man  ihre innere Struktur verstehen. Ein Agent besteht aus vier Hauptkomponenten, die zusammenarbeiten, um komplexe Aufgaben zu lösen. Diese Architektur ermöglicht es, die Stärken von LLMs mit praktischen Werkzeugen zu kombinieren.

**Die 4 Kern-Komponenten**

Jede Komponente eines Agenten hat eine spezifische Rolle im Gesamtsystem. Das Verständnis dieser Rollen hilft beim Design eigener Agenten und bei der Fehlersuche.

+ **LLM (Das Gehirn)**: Das Large Language Model fungiert als zentrale Intelligenz des Agenten. Es versteht die Benutzeranfrage, interpretiert Tool-Ergebnisse und entscheidet über nächste Schritte. Ohne das LLM wäre der Agent nur eine Sammlung unverbundener Werkzeuge.

+ **Tools (Die Hände)**: Tools sind spezialisierte Funktionen, die dem Agenten erlauben, mit der Außenwelt zu interagieren. Sie können so einfach sein wie ein Rechner oder so komplex wie eine Datenbankverbindung. Jedes Tool erweitert die Fähigkeiten des Agenten erheblich.

+ **Agent-Scratchpad (Das Gedächtnis)**: Das Scratchpad speichert Zwischenergebnisse und den Verlauf der Tool-Aufrufe. Es ermöglicht dem Agenten, aus vorherigen Schritten zu lernen und komplexe, mehrstufige Reasoning-Prozesse durchzuführen.

+ **Executor (Der Koordinator)**: Der Executor orchestriert das Zusammenspiel aller Komponenten. Er entscheidet, wann Tools aufgerufen werden, wann genug Informationen gesammelt wurden und wann die finale Antwort generiert werden soll.


<br>

**Der Agent-Denkprozess**

Der Denkprozess eines Agenten ist fundamental anders als der eines LLMs. Während ein LLM linear von Eingabe zu Ausgabe arbeitet, durchläuft ein Agent einen iterativen Zyklus aus Reasoning, Tool-Nutzung und Bewertung.

+ **Schritt 1 - Verstehen**: Der Agent analysiert die Benutzeranfrage und identifiziert, welche Informationen oder Aktionen benötigt werden. Dies geschieht durch das LLM, das die natürliche Sprache interpretiert.

+ **Schritt 2 - Planen**: Basierend auf dem Verständnis der Anfrage plant der Agent, welche Tools in welcher Reihenfolge verwendet werden sollten. Diese Planung kann sich während der Ausführung ändern.

+ **Schritt 3 - Ausführen**: Der Agent ruft das gewählte Tool auf und erhält ein Ergebnis. Dieses Ergebnis wird im Scratchpad gespeichert und steht für weitere Entscheidungen zur Verfügung.

+ **Schritt 4 - Bewerten**: Nach jedem Tool-Aufruf bewertet der Agent, ob genügend Informationen vorliegen oder weitere Schritte notwendig sind. Diese Bewertung bestimmt den weiteren Verlauf.

+ **Schritt 5 - Iterieren oder Antworten**: Je nach Bewertung kehrt der Agent zu Schritt 2 zurück oder generiert die finale Antwort. Diese Flexibilität ermöglicht die Lösung komplexer, unvorhersehbarer Probleme.



# 4 | Praktische Beispiele
---

Praktische Beispiele helfen dabei, die Konzepte von Agenten greifbar zu machen. Wir beginnen mit einfachen, fokussierten Agenten und steigern die Komplexität schrittweise. Diese Progression zeigt, wie Agenten für verschiedene Anwendungsfälle optimiert werden können.



## 4.1 Rechner-Agent

Ein Rechner-Agent demonstriert die Grundprinzipien der Agent-Architektur mit minimaler Komplexität. Er zeigt, wie ein Agent eine spezifische Domäne (Mathematik) abdecken kann, ohne von anderen Funktionen abgelenkt zu werden.


**🔢 RECHNER-AGENT DEMO**

**Konfiguration:** Agent nur mit Calculator-Tool ausgestattet

**Test-Fragen und Ergebnisse:**
- **"Was ist 15 * 23?"** → ✅ 345
- **"Berechne (100 + 50) * 3"** → ✅ 450  
- **"Was ist 2 hoch 10?"** → ✅ 1024

**Erkenntnisse:**
- **Fokussierung** auf eine Domäne macht den Agenten zuverlässiger
- **Wiederverwendbarkeit** derselben Tool-Logik für verschiedene Probleme
- **Transparenz** durch sichtbare Reasoning-Schritte


Dieser Agent zeigt drei wichtige Prinzipien: **Fokussierung** auf eine Domäne macht den Agenten zuverlässiger, **Wiederverwendbarkeit** derselben Tool-Logik für verschiedene Probleme, und **Transparenz** durch sichtbare Reasoning-Schritte.

## 4.2 Recherche-Agent

Ein Recherche-Agent erweitert die Fähigkeiten erheblich, indem er Zugang zu aktuellen Informationen erhält. Dies zeigt, wie Agenten die fundamentale Begrenzung von LLMs (veraltete Trainingsdaten) überwinden.


**🔍 RECHERCHE-AGENT DEMO**

**Konfiguration:** Agent nur mit Internet Search-Tool ausgestattet

**Test-Frage:** "Was sind die neuesten Nachrichten über künstliche Intelligenz?"

**Verhalten:**
- Agent ruft automatisch das Internet Search-Tool auf
- Durchsucht aktuelle Nachrichtenquellen
- Fasst relevante Informationen zusammen

**Beispiel-Ergebnis:**
"Aktuelle KI-Entwicklungen umfassen Durchbrüche in der Sprachverarbeitung, neue Robotik-Anwendungen in der Industrie und verstärkte Diskussionen über KI-Regulierung in Europa..."

**Key-Features:**
- **Aktualität** durch Zugriff auf Live-Daten
- **Validierung** von Informationen durch mehrere Quellen
- **Kontextualisierung** von Suchergebnissen für die spezifische Anfrage


Der Recherche-Agent demonstriert **Aktualität** durch Zugriff auf Live-Daten, **Validierung** von Informationen durch mehrere Quellen, und **Kontextualisierung** von Suchergebnissen für die spezifische Anfrage.



## 4.3 Multi-Tool Agent



Ein Multi-Tool Agent kombiniert verschiedene Fähigkeiten und zeigt die wahre Stärke der Agent-Architektur: die Orchestrierung mehrerer Tools zur Lösung komplexer, mehrdimensionaler Probleme.


**🛠️ MULTI-TOOL AGENT DEMO**

**Konfiguration:** Agent mit allen verfügbaren Tools (Internet, Rechner, Wetter)

**Komplexe Test-Anfrage:**
"Ich plane eine Reise nach München nächste Woche. Kannst du das aktuelle Wetter dort prüfen und berechnen was 3 Hotelübernachtungen à 120€ plus 2 Zugtickets à 89€ kosten würden?"

**Agent-Verhalten:**
1. **Wetter-Tool** → Aktuelle Wetterdaten für München abrufen
2. **Calculator-Tool** → Reisekosten berechnen: (3 × 120€) + (2 × 89€) = 538€
3. **Synthese** → Beide Informationen in kohärente Antwort zusammenfassen

**Beispiel-Antwort:**
"Das aktuelle Wetter in München beträgt 18°C mit teilweise bewölktem Himmel - ideal für eine Städtereise. Die Gesamtkosten für Ihre Reise belaufen sich auf 538€ (360€ für Hotels + 178€ für Zugtickets)."

**Besondere Fähigkeiten:**
- **Tool-Orchestrierung** durch intelligente Auswahl und Sequenzierung
- **Kontext-Erhaltung** zwischen verschiedenen Tool-Aufrufen
- **Synthesefähigkeit** beim Kombinieren verschiedener Informationstypen


Dieser Agent zeigt **Tool-Orchestrierung** durch intelligente Auswahl und Sequenzierung, **Kontext-Erhaltung** zwischen verschiedenen Tool-Aufrufen, und **Synthesefähigkeit** beim Kombinieren verschiedener Informationstypen.

# 5 | Wann braucht man einen Agent?
---

Die Entscheidung zwischen einem einfachen LLM und einem Agenten hängt von den spezifischen Anforderungen Ihrer Anwendung ab. Eine klare Entscheidungsmatrix hilft dabei, die richtige Technologie für den jeweiligen Anwendungsfall zu wählen und Ressourcen effizient einzusetzen.



Die Wahl der richtigen Technologie beginnt mit der Analyse der Aufgabenanforderungen. Während LLMs für viele Textverarbeitungsaufgaben ausreichen, sind Agenten unverzichtbar, wenn externe Interaktionen oder aktuelle Daten benötigt werden.

**Verwenden Sie einen Agenten wenn:**

+ Sie aktuelle oder dynamische Daten benötigen, die sich häufig ändern (Aktienkurse, Wetter, Nachrichten). Agenten können über APIs auf Live-Daten zugreifen und diese in ihre Antworten integrieren.

+ Präzise Berechnungen erforderlich sind, bei denen Genauigkeit kritisch ist. LLMs approximieren mathematische Operationen, während Agenten echte Rechner-Tools verwenden.

+ Externe Systeme angesprochen werden müssen, wie Datenbanken, APIs oder andere Services. Agenten können diese Integrationen nahtlos abwickeln.

+ Komplexe, mehrstufige Prozesse durchgeführt werden sollen, bei denen jeder Schritt vom vorherigen abhängt. Der Agent-Reasoning-Loop ist für solche Szenarien optimiert.

**Ein einfaches LLM reicht wenn:**

+ Reine Textverarbeitung ohne externe Daten im Fokus steht. Für Zusammenfassungen, Übersetzungen oder Textanalysen sind LLMs optimal.

+ Kreative Aufgaben gelöst werden sollen, wie das Schreiben von Geschichten, Gedichten oder Marketing-Texten. Hier sind die kreativen Fähigkeiten des LLMs gefragt.

+ Erklärungen oder Bildungsinhalt basierend auf allgemeinem Wissen benötigt werden. LLMs haben Zugang zu einem enormen Wissensfundus.

+ Statische Code-Generierung ohne externe Abhängigkeiten erforderlich ist. Für einfache Programmieraufgaben sind LLMs sehr effektiv.

<br>

**Die Faustregel lautet**:    
Wenn Sie Tools, aktuelle Daten oder externe Interaktionen benötigen, wählen Sie einen Agenten. Für reine Textverarbeitung reicht ein LLM aus.

# 6 | Hands-On: Agent bauen
---

In [ ]:
!uv pip install --system --prerelease allow -q tavily-python wikipedia

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain.utilities import WikipediaAPIWrapper
from tavily import TavilyClient
import os

In [ ]:
# Tools definieren

def read_file(filename):
    try:
        with open(filename, 'r') as f:
            return f.read()
    except:
        return f"Datei {filename} nicht gefunden"

def write_file(content):
    filename, text = content.split("|", 1)
    with open(filename, 'w') as f:
        f.write(text)
    return f"Datei {filename} geschrieben"

# Tavily Web-Suche
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))  # API-Key als Umgebungsvariable setzen

def web_search(query):
    try:
        result = tavily.search(query=query, search_depth="basic")
        return f"Suchergebnis: {result['results'][0]['content']}"
    except Exception as e:
        return f"Fehler bei Tavily-Suche: {e}"

# Wikipedia-Suche
def wiki_search(term):
    wiki = WikipediaAPIWrapper()
    try:
        return wiki.run(term)
    except Exception as e:
        return f"Wikipedia-Fehler: {e}"

custom_tools = [
    Tool(name="read_file", func=read_file, description="Datei lesen"),
    Tool(name="write_file", func=write_file, description="Datei schreiben (Format: datei.txt|inhalt)"),
    Tool(name="search", func=web_search, description="Führe eine Websuche mit Tavily durch"),
    Tool(name="wiki", func=wiki_search, description="Frage einen Begriff in Wikipedia nach")
]

In [ ]:
# Modell und ChatPrompt-Template definieren
model_name = "gpt-4o-mini"
temperature = 0.0

llm = ChatOpenAI(model=model_name, temperature=temperature)

# Prompt als Template definieren
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent mit Zugriff auf Tools."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# 🔧 Agenten-Logik
agent_runnable = create_tool_calling_agent(llm, custom_tools, prompt)

# Agenten erstellen
custom_agent = AgentExecutor(
    agent=agent_runnable,
    tools=custom_tools,
    verbose=True,
)

In [ ]:
# 🔧 Test
input = """
Erstelle eine Datei 'notiz.txt' mit dem Inhalt 'Agenten können autonom agieren. 🤖',
lies sie dann wieder,
suche nach 'Python Programmierung' im Web.
was steht zu Taylor Swift auf Wikipedia
"""

response = custom_agent.invoke({"input": input})

In [ ]:
mprint("## 🛠️ Hands-On Agent")
mprint("---")
mprint("**Input:**")
mprint(response['input'])
mprint("**Output**:")
mprint(response['output'])

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Kalkulation
</font></p>

Gegeben ist eine Datei, die eine Reihe von Gleichungen enthält.
Der Dateiname ist GenAI/02 data/gleichungen.txt

**Gleichung:**    
41748459 - 87226336    
92995162 * 46769739    
61530438 * 56074589    
95329602 + 45418854    
412907 + 3731910    
...

Verwenden Sie einen LangChain-Agenten mit einem Tool, um jede dieser Gleichungen zu berechnen, und erstellen Sie eine Datei ähnlich dieser:

**Ergebnisse:**  
41748459 - 87226336 = 45477877   
92995162 * 46769739 = 4349359455002718   
61530438 * 56074589 = 3450294021839982   
95329602 + 45418854 = 140748456   
412907 + 3731910 = 4144817   
... ...



# B | Model Context Protocol (MCP)
---



**Wichtige Ressourcen:**

[Anthropic MCP](https://www.anthropic.com/news/model-context-protocol)

[OpenAI MCP](https://openai.github.io/openai-agents-python/mcp/)

[MCPServer](https://github.com/modelcontextprotocol/servers)




<p><font color='black' size="5">
5.1 Was ist MCP?
</font></p>


Ein Protokoll ist ein Regelwerk, das bestimmt, wie zwei Systeme miteinander kommunizieren. Protokolle regeln die Datenübertragung in Computernetzwerken, bei der Internetkommunikation und zwischen Softwaresystemen.

**Zum Beispiel:**

+ HTTP (Hypertext Transfer Protocol): Ermöglicht Websites die Kommunikation mit Browsern.
+ TCP/IP (Transmission Control Protocol/Internetprotokoll): Definiert, wie Datenpakete im Internet geroutet werden.
+ JSON-RPC (Aufrufen von Remoteprozeduren): Ein Protokoll, das den Datenaustausch im JSON-Format ermöglicht.    

Das **Model Context Protocol (MCP)** ist ein offenes Protokoll, das es großen Sprachmodellen (LLMs) ermöglicht, sich auf standardisierte Weise in externe Datenquellen und Tools zu integrieren. Dieses von Anthropic entwickelte Protokoll macht es KI-Modellen leicht, nahtlos mit einer Vielzahl von Tools und Datenquellen zusammenzuarbeiten.

Es soll die Interaktion von KI-Modellen, insbesondere Large Language Models (LLMs) und autonomen Agenten, mit externen Datenquellen und Tools zu **standardisieren und zu vereinfachen**. Ziel ist es, einen **einheitlichen Rahmen** zu schaffen, der es KI-Agenten ermöglicht, auf strukturierte Daten aus verschiedenen Quellen wie Datenbanken, APIs, Cloud-Speicher und Unternehmensanwendungen auf standardisierte Weise zuzugreifen, diese zu verarbeiten und darauf zu reagieren, **ohne dass für jede Quelle spezifische API-Integrationen erforderlich sind**.




<p><font color='black' size="5">
5.2 Warum wurde MCP entwickelt?
</font></p>



Die Notwendigkeit von MCP ergibt sich aus den **Ineffizienzen und Herausforderungen** aktueller KI-API-Interaktionen. Derzeit ist der Aufbau von KI-Agenten, die Daten aus verschiedenen Quellen abrufen, **fragmentiert, repetitiv und schwer zu skalieren**. Jedes Tool spricht seine eigene Sprache und erfordert **individuelle Integrationen**. MCP zielt darauf ab, diese Komplexität zu reduzieren und den **Entwicklungsaufwand zu minimieren**.




<p><font color='black' size="5">
5.3 Wie funktioniert MCP?
</font></p>



MCP basiert auf einer **Client-Server-Architektur**.

*   **MCP-Clients** sind typischerweise KI-Agenten, Anwendungen oder Systeme, die strukturierte Daten benötigen. Beispiele hierfür sind Claude Desktop, Cursor, Windsurf und Frameworks wie Langchain und Pydantic AI.
*   **MCP-Server** fungieren als **Vermittler**, die Daten von verschiedenen APIs, Datenbanken oder Unternehmenssystemen abrufen und diese in einem **einheitlichen Format** an die Clients zurückgeben.

Der Interaktionsprozess folgt einem strukturierten **Anfrage-Antwort-Zyklus**:

1.  Der KI-Agent (Client) sendet eine **strukturierte Anfrage** an den MCP-Server, in der die benötigten Daten oder die auszuführende Aktion in einem standardisierten Format definiert sind.
2.  Der MCP-Server **verarbeitet** diese Anfrage, authentifiziert sie, prüft die Berechtigungen und ermittelt, welche externen Systeme abgefragt werden müssen.
3.  Die eigentlichen **Datenabfragen** an die verschiedenen Quellen können parallel erfolgen.
4.  Die **Antworten** der verschiedenen Quellen werden vom MCP-Server in einem **einheitlichen, strukturierten Format standardisiert**, das für KI-Modelle leicht zu verarbeiten ist.

Ein wesentliches Konzept von MCP ist die **Reflection**. Dies bedeutet, dass ein MCP-Client einen MCP-Server nach seinen **verfügbaren Tools und Ressourcen** fragen kann, ohne vorherige Kenntnisse dieser Schnittstellen zu benötigen.

[Interaktive Visualisierung des MCP-Prozesses](https://claude.site/artifacts/1ba26344-3819-427d-9080-98381785f8df)




<p><font color='black' size="5">
5.4 Kernkonzepte
</font></p>



*   **Tools:** Funktionen, die das Modell nutzen kann, um Aktionen durchzuführen (z. B. Websuchen, Datenbankabfragen).
*   **Ressourcen:** Anhänge oder Daten, die dem Modell zur Verfügung gestellt werden (z. B. Dateien, Datenbankinhalte).
*   **Prompts:** Vorlagen, die Clients für Anfragen an das Modell verwenden können.




<p><font color='black' size="5">
5.5 Vorteile von MCP
</font></p>


*   **Vereinfachte Integrationen** und **reduzierte Komplexität**.
*   **Verbesserte Skalierbarkeit** und **Wiederverwendbarkeit** von Integrationen.
*   **Erhöhte Interoperabilität** zwischen KI-Modellen und externen Systemen.
*   **Zeitersparnis für Entwickler** durch weniger benutzerdefinierte API-Implementierungen.
*   Potenzial für eine **zentrale Authentifizierung** (zukünftig).

[Interaktive Visualisierung](https://claude.site/artifacts/1ba26344-3819-427d-9080-98381785f8df)

<p><font color='black' size="5">
Code-Beispiel zur Interaktion
</font></p>

In [ ]:
import json
import uuid
from typing import Dict, Any

class MCPClient:
    """Einfacher MCP-Client für die Kommunikation mit MCP-Servern."""

    def __init__(self, client_id: str, api_key: str):
        self.client_id = client_id
        self.api_key = api_key
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Client-ID": client_id
        }

    def create_request(self, tool_name: str, parameters: Dict[str, Any]) -> Dict[str, Any]:
        """Erstellt eine strukturierte MCP-Anfrage."""
        request_data = {
            "protocol_version": "1.0",
            "request_id": str(uuid.uuid4()),
            "tool": {
                "name": tool_name,
                "parameters": parameters
            }
        }
        return request_data

    def send_request(self, request_data: Dict[str, Any]) -> Dict[str, Any]:
        """Sendet eine MCP-Anfrage an den MCP-Server."""
        # In einer realen Implementierung würde hier ein HTTP-Request erfolgen
        return {"status": "success", "result": {}}  # Platzhalter


class MCPServer:
    """Simulierter MCP-Server zum Testen der Interaktion."""

    def __init__(self):
        self.registered_tools = {
            "weather_info": self._get_weather_data,
            "database_query": self._execute_database_query,
            "text_translation": self._translate_text
        }

    def validate_request(self, request: Dict[str, Any]) -> bool:
        """Überprüft, ob die Anfrage dem MCP-Protokoll entspricht."""
        required_fields = ["protocol_version", "request_id", "tool"]
        if not all(field in request for field in required_fields):
            return False

        if "name" not in request["tool"] or request["tool"]["name"] not in self.registered_tools:
            return False

        return True

    def process_request(self, request: Dict[str, Any]) -> Dict[str, Any]:
        """Verarbeitet eine validierte MCP-Anfrage."""
        if not self.validate_request(request):
            return {"error": "Ungültige Anfrage", "status": "error"}

        tool_name = request["tool"]["name"]
        parameters = request["tool"].get("parameters", {})

        # Tool ausführen
        tool_func = self.registered_tools[tool_name]
        result = tool_func(parameters)

        # Antwort standardisieren
        response = {
            "request_id": request["request_id"],
            "status": "success",
            "result": result
        }
        return response

    def _get_weather_data(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Simuliert den Abruf von Wetterdaten von einem externen Dienst."""
        return {
            "city": params.get("city", "Berlin"),
            "temperature": 22,
            "condition": "sonnig",
            "humidity": 65
        }

    def _execute_database_query(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Simuliert eine Datenbankabfrage."""
        return {
            "rows": [
                {"id": 1, "name": "Produkt A", "price": 29.99},
                {"id": 2, "name": "Produkt B", "price": 49.99}
            ],
            "total_rows": 2
        }

    def _translate_text(self, params: Dict[str, Any]) -> Dict[str, Any]:
        """Simuliert einen Übersetzungsdienst."""
        target_lang = params.get("target_language", "en")

        translations = {
            "en": "Hello world",
            "fr": "Bonjour le monde",
            "de": "Hallo Welt"
        }

        return {
            "original_text": params.get("text", ""),
            "translated_text": translations.get(target_lang, "Übersetzung nicht verfügbar"),
            "target_language": target_lang
        }


class LLMApplication:
    """LLM-Anwendung, die den MCP-Client verwendet."""

    def __init__(self, mcp_client: MCPClient):
        self.mcp_client = mcp_client

    def process_user_query(self, query: str) -> str:
        """Verarbeitet eine Nutzeranfrage mit MCP für zusätzliche Daten."""
        if "wetter" in query.lower():
            # Extrahiere Stadt (vereinfacht)
            city = "Berlin"

            # Erstelle und sende MCP-Anfrage
            request = self.mcp_client.create_request("weather_info", {"city": city})
            response = self.mcp_client.send_request(request)

            if response.get("status") == "success":
                weather_data = response.get("result", {})
                return self._format_weather_response(weather_data)
            return "Entschuldigung, ich konnte keine Wetterdaten abrufen."

        elif "übersetze" in query.lower():
            text = "Hallo, wie geht es dir?"
            target_lang = "en"

            request = self.mcp_client.create_request("text_translation", {
                "text": text,
                "target_language": target_lang
            })
            response = self.mcp_client.send_request(request)

            if response.get("status") == "success":
                translation_data = response.get("result", {})
                return self._format_translation_response(translation_data)
            return "Entschuldigung, ich konnte den Text nicht übersetzen."

        return "Ich verstehe Ihre Anfrage. Wie kann ich Ihnen helfen?"

    def _format_weather_response(self, weather_data: Dict[str, Any]) -> str:
        """Formatiert Wetterdaten für die Anzeige."""
        return (f"Das aktuelle Wetter in {weather_data.get('city')}: "
                f"{weather_data.get('temperature')}°C, {weather_data.get('condition')}.")

    def _format_translation_response(self, translation_data: Dict[str, Any]) -> str:
        """Formatiert Übersetzungsdaten für die Anzeige."""
        return (f"Übersetzung ({translation_data.get('target_language')}): "
                f"{translation_data.get('translated_text')}")


def run_example():
    """Führt ein Beispiel der MCP-Interaktion aus."""
    print("=== MCP-Interaktionsbeispiel ===\n")

    # MCP-Server initialisieren
    server = MCPServer()

    # MCP-Client initialisieren
    client = MCPClient(
        client_id="llm_app_123",
        api_key="sk_test_12345"
    )

    # LLM-Anwendung initialisieren
    llm_app = LLMApplication(client)

    # Mock für send_request
    def mock_send_request(request_data):
        print("\n[1] LLM-Anwendung erstellt MCP-Anfrage:")
        print(json.dumps(request_data, indent=2))

        print("\n[2] MCP-Client sendet Anfrage an MCP-Server")

        print("\n[3] MCP-Server validiert die Anfrage")
        valid = server.validate_request(request_data)
        print(f"Anfrage ist gültig: {valid}")

        print("\n[4] MCP-Server verarbeitet die Anfrage und kommuniziert mit externen Systemen")
        response = server.process_request(request_data)

        print("\n[5] Externe Systeme liefern Daten an MCP-Server")

        print("\n[6] MCP-Server standardisiert die Antwort:")
        print(json.dumps(response, indent=2))

        print("\n[7] MCP-Server sendet Antwort an MCP-Client")

        print("\n[8] MCP-Client stellt Kontext der LLM-Anwendung zur Verfügung")

        return response

    # Überschreiben der Methode
    client.send_request = mock_send_request

    # Beispiel-Nutzeranfrage
    query = "Wie ist das Wetter in Berlin?"

    print(f"\nNutzer fragt: '{query}'")
    print("\n[0] LLM-Anwendung analysiert die Nutzeranfrage")

    # LLM verarbeitet die Anfrage und nutzt MCP
    response = llm_app.process_user_query(query)

    print("\n[9] LLM-Anwendung nutzt den Kontext zur Beantwortung")
    print(f"\nAntwort an den Nutzer: '{response}'")


if __name__ == "__main__":
    run_example()

<p><font color='black' size="5">
Fazit
</font></p>





Das Model Context Protocol (MCP) stellt einen vielversprechenden Ansatz dar, um die Integration von KI-Modellen mit der Außenwelt zu **standardisieren und zu vereinfachen**. Durch die Definition eines gemeinsamen Protokolls und die Bereitstellung von Konzepten wie Tools und Ressourcen ermöglicht MCP die Entwicklung **flexiblerer, skalierbarer und wiederverwendbarer** KI-Anwendungen. Obwohl es noch Herausforderungen zu bewältigen gibt, hat MCP das Potenzial, die Art und Weise, wie KI-Agenten mit Daten und Tools interagieren, **grundlegend zu verändern**.